## Part 2: Training your own ML Model

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part2/train_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Read contents of files

will take a folder as an input, read the files in this folder and store their contents as a Python list data structure

In [2]:
import os
import codecs

def get_curr_dir():
    return os.getcwd().replace("\\", "/")+"/"
    
def read_in(folder):
    files = os.listdir(folder)
    a_list = []
    for a_file in files:
        if not a_file.startswith("."):
            f = codecs.open(folder + a_file, 
                "r", encoding = "ISO-8859-1", errors="ignore")
            a_list.append(f.read())
            f.close()
    return a_list

spam_list = read_in(get_curr_dir()+"enron1/spam/")
ham_list = read_in(get_curr_dir()+"enron1/ham/")
print(len(spam_list))
print(len(ham_list))

1500
3672


### Combine data into single structure

In [3]:
import random
 
all_emails = [(email_content, "spam") for email_content in spam_list]
all_emails += [(email_content, "ham") for email_content in ham_list]
random.seed(42)
random.shuffle(all_emails)
print (f"Dataset size = {str(len(all_emails))} emails")

Dataset size = 5172 emails


from text, get features by tokenizing and making all words lowercase


In [4]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

stopwords_eng = stopwords.words('english')

def get_features(text):
    tokenized_list = [w.lower() for w in word_tokenize(text) if w not in stopwords_eng and w not in punctuation]
    features = {}
    for word in tokenized_list:
        features[word] = True
    return features
    
all_features = [(get_features(email), label) 
                 for (email, label) in all_emails]
#index of all_features is the email index
#index 0 of tuple is features (dictionary of words) of the email
#index 1 of tuple is the label spam or ham

In [5]:
from nltk import NaiveBayesClassifier, classify
 
def train(features, proportion):
    train_size = int(len(features) * proportion)
    train_set, test_set = features[:train_size], features[train_size:]
    print (f"Training set size = {str(len(train_set))} emails")
    print (f"Test set size = {str(len(test_set))} emails")
    classifier = NaiveBayesClassifier.train(train_set)
    return train_set, test_set, classifier
 
train_set, test_set, classifier = train(all_features, 0.75)

Training set size = 3879 emails
Test set size = 1293 emails


In [7]:
print(classifier.classify(get_features("Welcome to you burial insurance alert!")))
print(classifier.classify(get_features("your email has won $1,000,000!!!!!!")))
print(classifier.classify(get_features("Confirmation 05430")))
print(classifier.classify(get_features("Board meeting August 10")))
print(classifier.classify(get_features("Preparing for the fall semester")))
print(classifier.classify(get_features("Free webinar: run your programs faster anywhere")))

spam
spam
ham
ham
ham
spam


In [16]:
def evaluate(train_set, test_set, classifier):
    print (f"Accuracy on the training set = {str(classify.accuracy(classifier, train_set))}")
    print (f"Accuracy of the test set = {str(classify.accuracy(classifier, test_set))}")
    classifier.show_most_informative_features(50)
 
#evaluate(train_set, test_set, classifier)

Accuracy on the training set = 0.9582366589327146
Accuracy of the test set = 0.9404485692188709
Most Informative Features
               forwarded = True              ham : spam   =    184.6 : 1.0
                    2004 = True             spam : ham    =    136.8 : 1.0
            prescription = True             spam : ham    =    117.4 : 1.0
                     nom = True              ham : spam   =    117.1 : 1.0
                    pain = True             spam : ham    =     93.1 : 1.0
                  health = True             spam : ham    =     81.8 : 1.0
                    2001 = True              ham : spam   =     75.2 : 1.0
                featured = True             spam : ham    =     73.7 : 1.0
                thousand = True             spam : ham    =     72.0 : 1.0
                     ect = True              ham : spam   =     71.0 : 1.0
             medications = True             spam : ham    =     68.8 : 1.0
              nomination = True              ham : sp

### Define feature extractor and bag-of-words converter

Given a list of (already tokenized) words, we need a function to extract just the ones we care about: those not found in the list of English stopwords or standard punctuation.

We also need a way to easily turn a list of words into a [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model), pairing each word with the count of its occurrences.

In [3]:
from nltk.corpus import stopwords
from string import punctuation

stopwords_eng = stopwords.words('english')

def extract_features(words):
    return [w for w in words if w not in stopwords_eng and w not in punctuation]

def bag_of_words(words):
    bag = {}
    for w in words:
        bag[w] = bag.get(w,0)+1
    return bag

### Ingest, clean, and convert the positive and negative reviews

For both the positive ("pos") and negative ("neg") sets of reviews, extract the features and convert to bag of words. From these, we construct a list of tuples known as a "featureset": the first part of each tuple is the bag of words for that review, and the second is its label ("pos"/"neg").

Note that `movie_reviews.words(fileid)` provides a tokenized list of words. If we wanted the un-tokenized text, we would use `movie_reviews.raw(fileid)` instead, then tokenize it using our preferred tokenizeer (e.g. [nltk.tokenize.word_tokenize](https://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.punkt.PunktLanguageVars.word_tokenize)).

In [4]:
from nltk.corpus import movie_reviews

reviews_pos = []
reviews_neg = []
for fileid in movie_reviews.fileids('pos'):
    words = extract_features(movie_reviews.words(fileid))
    reviews_pos.append((bag_of_words(words), 'pos'))
for fileid in movie_reviews.fileids('neg'):
    words = extract_features(movie_reviews.words(fileid))
    reviews_neg.append((bag_of_words(words), 'neg'))

### Split reviews into training and test sets
We need to break up each group of reviews into a training set (about 80%) and a test set (the remaining 20%). In case there's some meaningful order to the reviews (e.g. the first 800 are from one group of reviewers, the next 200 are from another), we shuffle the sets first to ensure we aren't introducing additional bias. Note that this means our accuracy will not be exactly the same on every run; if you wish to see consistent results on each run, you can stabilize the shuffle by calling [random.seed(n)](https://www.geeksforgeeks.org/random-seed-in-python/) first.

In [5]:
from random import shuffle

split_pct = .80

def split_set(review_set):
    split = int(len(review_set)*split_pct)
    return (review_set[:split], review_set[split:])

shuffle(reviews_pos)
shuffle(reviews_neg)

pos_train, pos_test = split_set(reviews_pos)
neg_train, neg_test = split_set(reviews_neg)

train_set = pos_train+neg_train
test_set = pos_test+neg_test

### Train the model

Now that our data is ready, the training step itself is quite simple if we use the [NaiveBayesClassifier](https://www.nltk.org/api/nltk.classify.html#module-nltk.classify.naivebayes) provided by NLTK.

If you are used to methods such as `model.fit(x,y)` which take two parameters -- the data and the labels -- it may be confusing that `NaiveBayesClassifier.train` takes just one argument. This is because the labels are already embedded in `train_set`: each element in the set is a Bag of Words paired with a 'pos' or 'neg'; value.

In [6]:
from nltk.classify import NaiveBayesClassifier

model = NaiveBayesClassifier.train(train_set)

### Check model accuracy

NLTK's built-in [accuracy](https://www.nltk.org/api/nltk.classify.html#module-nltk.classify.util) utility can run our test_set through the model and compare the labels returned by the model to the labels in the test set, producing an overall % accuracy. Not too impressive, right? We need to improve.

In [7]:
from nltk.classify.util import accuracy

print(100 * accuracy(model, test_set))

75.0


### Save the model
Our trained model will be cleared from memory when this notebook is closed. So that we can use it again later, save the model as a file using the [pickle](https://docs.python.org/3/library/pickle.html) serializer.

In [ ]:
import pickle

model_file = open('sa_classifier.pickle','wb')
pickle.dump(model, model_file)
model_file.close()

### Save the model (Colab version)

Google Colab doesn't provide direct access to files saved during a notebook session, so we need to save it in [Google Drive](https://drive.google.com) instead. The first time you run this, it will ask for permission to access your Google Drive. Follow the instructions, then wait a few minutes and look for a new folder called "Colab Output" in [Drive](https://drive.google.com). Note that Colab does not alway sync to Drive immediately, so check the file update times and re-run this cell if it doesn't look like you have the most revent version of your file.

In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/gdrive')
    !mkdir -p '/content/gdrive/My Drive/Colab Output'
    model_file = open('/content/gdrive/My Drive/Colab Output/sa_classifier.pickle','wb')
    pickle.dump(model, model_file)
    model_file.flush()
    print('Model saved in /content/gdrive/My Drive/Colab Output')
    !ls '/content/gdrive/My Drive/Colab Output'
    drive.flush_and_unmount()
    print('Re-run this cell if you cannot find it in https://drive.google.com')